## Ingest constructors.json file

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
%run "../includes/configurations"

In [0]:
%run "../includes/common_functions"

**Step 1 : Read the JSON file using the spark dataframe reader**

In [0]:
display(dbutils.fs.ls(f'{raw_folder_path}'))

In [0]:
constructor_schema = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

constructor_df = spark.read.schema(constructor_schema) \
    .json(f"{raw_folder_path}/constructors.json")

constructor_df.limit(5).display()

In [0]:
display(constructor_df.printSchema())

In [0]:
## step 2: drop the unwanted columns from the dataframe

constructor_dropped_df = constructor_df.drop(constructor_df['url'])


In [0]:
from pyspark.sql.functions import col
constructor_dropped_df = constructor_df.drop(col('url'))

In [0]:
from pyspark.sql.functions import current_timestamp, lit
constructor_renamed_df = constructor_dropped_df.withColumnRenamed('constructorId', 'constructor_id') \
                                               .withColumnRenamed('constructorRef', 'constructor_ref') \
                                                   .withColumn('data_source', lit(v_data_source))

constructor_final_df = constructor_renamed_df.withColumn('ingestion_date', current_timestamp())

constructor_final_df = ingestion_date(constructor_renamed_df)

constructor_final_df.limit(5).display()

In [0]:
constructor_final_df.write.mode('overwrite').format("parquet").saveAsTable("f1_processed.constructors")

In [0]:
# check whether the file is in the assigned folder or not
display(dbutils.fs.ls(f'{processed_folder_path}/constructors'))

In [0]:
dbutils.notebook.exit("Success")